In [33]:
#=
The MIT License (MIT)

Copyright © 2023 Dr Keith S Reid CAilleach Computing Ltd

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated 
documentation files (the “Software”), to deal in the Software without restriction, including without 
limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the 
Software, and to permit persons to whom the Software is furnished to do so, subject to the following 
conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions 
of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED 
TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL 
THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF 
CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER 
DEALINGS IN THE SOFTWARE.

=#

#=

Julia Version 1.8.1 (2022-09-06)

Date 18 Feb 2023

Box Spec

            .-/+oossssoo+/-.               
        `:+ssssssssssssssssss+:`           ------------ 
      -+ssssssssssssssssssyyssss+-         OS: Ubuntu 22.04.1 LTS x86_64 
    .ossssssssssssssssssdMMMNysssso.       Host: 
   /ssssssssssshdmmNNmmyNMMMMhssssss/      Kernel: 5.15.0-60-generic 
  +ssssssssshmydMMMMMMMNddddyssssssss+     Uptime: 
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/    Packages: 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Shell: bash 5.1.16 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   Resolution: 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   DE: GNOME 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   WM: Mutter 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   WM Theme: Adwaita 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Theme: 
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/    Icons: 
  +sssssssssdmydMMMMMMMMddddyssssssss+     Terminal: 
   /ssssssssssshdmNNNNmyNMMMMhssssss/      CPU: AMD Ryzen 9 3900X (24) @ 3.800G 
    .ossssssssssssssssssdMMMNysssso.       GPU: 
      -+sssssssssssssssssyyyssss+-         Memory: 64Gb RAM
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.                                       
                                                                   

=#

#=
Intent:
Implement Sterne Broco in Python and Julia using TDD and nothing too clever
For speed comparison
Roughly simlar logic in the two versions
=#

In [34]:
# 1 Packages

using Test

In [35]:
# 2 Structs

mutable struct RationalNode
    numerator::Int64
    denominator::Int64
    place::Int64
    layer::Int64
end

In [36]:
# 3 Config

function get_depth()
    depth::Int64 = 3
    return depth
end

get_depth (generic function with 1 method)

In [37]:
# 4 model

function count_nodes(depth)
    node_count::Int64 = (2^depth)-1 
    return node_count
end

count_nodes (generic function with 1 method)

In [38]:
function init_tree(node_count, this_power)
    # these nodes predate root and are loop invariants
    zero_over_one   = RationalNode(0,1,0,           0)
    one_over_zero   = RationalNode(1,0,node_count+1,0)

    # cohering the branches
    triangular_part = [RationalNode(0,0,i,1) for i in 1:node_count]
    tree            = [zero_over_one]
    tree = vcat(tree,triangular_part)
    tree = vcat(tree,one_over_zero)
    
    # child logic
    left_parent         = tree[this_power+this_power+1]
    right_parent        = tree[this_power-this_power+1]
    child_place         = this_power
    child_numerator     = left_parent.numerator   + right_parent.numerator
    child_denominator   = left_parent.denominator + right_parent.denominator
    child               = RationalNode(child_numerator,child_denominator,child_place,1)
    tree[child.place+1] = child
    child_places        = [child.place]

    return tree, child_places
end


init_tree (generic function with 1 method)

In [39]:
function build_tree()
    
    #over_power::Int64           = 2^depth
    #this_power::Int64           = over_power/2
    #child_places::Array{Int64}  = []
    #tree::Array{RationalNode}   = []
        
    # tree has configurable depth see config
    depth::Int64            = get_depth()
    node_count::Int64       = count_nodes(depth)
    powers::Array{Int64}    = [2^x for x in (depth-1):-1:0]
    this_power              = powers[1]    
    tree, child_places      = init_tree(node_count,this_power)

    for current_layer in 1:depth
        this_power = powers[current_layer]    
        grand_child_places  = []
        for parent_place in child_places
            left_child_place    =  parent_place - this_power 
            right_child_place   =  parent_place + this_power
            append!(grand_child_places, [left_child_place, right_child_place])
        end
        child_places = grand_child_places
        for this_child_place in child_places
            left_parent         = tree[this_power+this_power+1]
            right_parent        = tree[this_power-this_power+1]
            child_place         = this_power
            child_numerator     = left_parent.numerator   + right_parent.numerator
            child_denominator   = left_parent.denominator + right_parent.denominator
            child               = RationalNode(child_numerator,child_denominator,child_place,current_layer)
            tree[child.place+1] = child
        end
    end
    return tree
end

build_tree (generic function with 1 method)

In [40]:
# 5 view

function draw_tree(tree)
    depth       = get_depth()
    over_power  = 2^depth
    height      = 1+depth
    width       = 1+over_power
    numerator_host   = zeros(height,width)
    denominator_host = zeros(height,width)

    num_printable = ""
    den_printable = ""
    
    for node in tree
        numerator_host[node.layer][node.place]  = node.numerator
        denominator_host[node.layer][node.place]= node.denominator
    end

    for num_row in enumerate(numerator_host)
        both = zip(num_row[2],denominator_host[num_row[1]])
        num_printable = ""
        den_printable = ""
        for x in both
            if x == (0.0,0.0)
                num_printable = num_printable*(" ")
                den_printable = den_printable*(" ")
            else
                num_printable = num_printable*(str(int(x[1])))
                den_printable = den_printable*(str(int(x[2])))
            end
        end
    end
    print(num_printable)
    print(den_printable)
end
    

draw_tree (generic function with 1 method)

In [41]:
# 6 control

function main_sb()
    tree        = build_tree()
    
    for this_node in tree
        println(this_node)
    end
    
#    draw_tree(tree)


#    depth           = get_depth()
 #   average_speed   = @time(build_tree)
  #  print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
   # print("At a depth of ", depth, " layers.")
end

main_sb();

RationalNode(0, 1, 0, 0)
RationalNode(1, 3, 1, 3)
RationalNode(1, 2, 2, 2)
RationalNode(0, 0, 3, 1)
RationalNode(1, 1, 4, 1)
RationalNode(0, 0, 5, 1)
RationalNode(0, 0, 6, 1)
RationalNode(0, 0, 7, 1)
RationalNode(1, 0, 8, 0)


In [32]:
function test_count_nodes()
    depth = 0
    node_count = count_nodes(depth)    
    @test (node_count == 0) 
    
    depth = 1
    node_count = count_nodes(depth)
    @test (node_count == 1) 
    
    depth = 2
    node_count = count_nodes(depth)
    @test (node_count == 3) 
    
    depth = 10
    node_count = count_nodes(depth)
    @test (node_count == 1023) 
    
    depth = 21
    node_count = count_nodes(depth)
    @test (node_count == 2097151)
    
    println("passed test count nodes")
end

function test_init_tree()
    depth::Int64            = 1
    node_count::Int64       = count_nodes(depth)
    powers::Array{Int64}    = [2^x for x in (depth-1):-1:0]
    this_power              = powers[1]    
    tree, child_places      = init_tree(node_count,this_power)
    
    @test child_places == [1]
    @test length(tree) == 3
    
    @test tree[1].numerator    == 0
    @test tree[1].denominator  == 1
    @test tree[1].place        == 0
    @test tree[1].layer        == 0
    
    @test tree[2].numerator    == 1
    @test tree[2].denominator  == 1
    @test tree[2].place        == 1
    @test tree[2].layer        == 1
    
    @test tree[3].numerator    == 1
    @test tree[3].denominator  == 0
    @test tree[3].place        == 2
    @test tree[3].layer        == 0
    
    depth                   = 2
    node_count              = count_nodes(depth)
    powers                  = [2^x for x in (depth-1):-1:0]
    this_power              = powers[1]    
    tree, child_places      = init_tree(node_count,this_power)

    @test child_places == [2]
    @test length(tree) == 5
    
    @test tree[1].numerator    == 0
    @test tree[1].denominator  == 1
    @test tree[1].place        == 0
    @test tree[1].layer        == 0
    
    @test tree[3].numerator    == 1
    @test tree[3].denominator  == 1
    @test tree[3].place        == 2
    @test tree[3].layer        == 1
    
    @test tree[5].numerator    == 1
    @test tree[5].denominator  == 0
    @test tree[5].place        == 4
    @test tree[5].layer        == 0
    
    depth                   = 5
    node_count              = count_nodes(depth)
    powers                  = [2^x for x in (depth-1):-1:0]
    this_power              = powers[1]    
    tree, child_places      = init_tree(node_count,this_power)

    @test child_places == [16]
    @test length(tree) == 33
    
    @test tree[1].numerator    == 0
    @test tree[1].denominator  == 1
    @test tree[1].place        == 0
    @test tree[1].layer        == 0
    
    @test tree[17].numerator    == 1
    @test tree[17].denominator  == 1
    @test tree[17].place        == 16
    @test tree[17].layer        == 1
    
    @test tree[33].numerator    == 1
    @test tree[33].denominator  == 0
    @test tree[33].place        == 32
    @test tree[33].layer        == 0
    
    println("passed init tree")
end


function JuliaSternBrocotTests()
    test_count_nodes()
    test_init_tree()
    print("passed all tests")
end

JuliaSternBrocotTests()

passed test count nodes
RationalNode(0, 1, 0, 0)
RationalNode(0, 0, 1, 1)
RationalNode(0, 0, 2, 1)
RationalNode(0, 0, 3, 1)
RationalNode(0, 0, 4, 1)
RationalNode(0, 0, 5, 1)
RationalNode(0, 0, 6, 1)
RationalNode(0, 0, 7, 1)
RationalNode(0, 0, 8, 1)
RationalNode(0, 0, 9, 1)
RationalNode(0, 0, 10, 1)
RationalNode(0, 0, 11, 1)
RationalNode(0, 0, 12, 1)
RationalNode(0, 0, 13, 1)
RationalNode(0, 0, 14, 1)
RationalNode(0, 0, 15, 1)
RationalNode(1, 1, 16, 1)
RationalNode(0, 0, 17, 1)
RationalNode(0, 0, 18, 1)
RationalNode(0, 0, 19, 1)
RationalNode(0, 0, 20, 1)
RationalNode(0, 0, 21, 1)
RationalNode(0, 0, 22, 1)
RationalNode(0, 0, 23, 1)
RationalNode(0, 0, 24, 1)
RationalNode(0, 0, 25, 1)
RationalNode(0, 0, 26, 1)
RationalNode(0, 0, 27, 1)
RationalNode(0, 0, 28, 1)
RationalNode(0, 0, 29, 1)
RationalNode(0, 0, 30, 1)
RationalNode(0, 0, 31, 1)
RationalNode(1, 0, 32, 0)
passed init tree
passed all tests